In [ ]:
import copy
import itertools
import numpy as np
import tensorflow as tf
from tensorflow.nn import softmax
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, SimpleRNNCell,Flatten
from tensorflow.keras import Input
from tensorflow.keras.layers import RNN, SimpleRNNCell, LSTMCell, GRUCell

def dRNN(cell, inputs, rate, scope='default'):

    n_steps = len(inputs)
    if rate < 0 or rate >= n_steps:
        raise ValueError('The \'rate\' variable needs to be adjusted.')
    print("Building layer: %s, input length: %d, dilation rate: %d, input dim: %d." % (scope, n_steps, rate, inputs[0].shape[1]))

    # make the length of inputs divide 'rate', by using zero-padding
    EVEN = (n_steps % rate) == 0
    if not EVEN:
        # Create a tensor in shape (batch_size, input_dims), which all elements are zero.
        # This is used for zero padding
        zero_tensor = tf.zeros_like(inputs[0])
        dialated_n_steps = n_steps // rate + 1
        print("=====> %d time points need to be padded. " % (dialated_n_steps * rate - n_steps))
        print("=====> Input length for sub-RNN: %d" % (dialated_n_steps))
        for i_pad in range(dialated_n_steps * rate - n_steps):
            inputs.append(zero_tensor)
    else:
        dialated_n_steps = n_steps // rate
        print("=====> Input length for sub-RNN: %d" % (dialated_n_steps))


    dilated_inputs = [tf.concat(inputs[i * rate:(i + 1) * rate], axis=0) for i in range(dialated_n_steps)]


    # building a dilated RNN with reformatted (dilated) inputs
    dilated_outputs = tf.keras.layers.RNN(cell, return_sequences=True, return_state=False, go_backwards=False,
                                          stateful=False, time_major=True, unroll=False, input_shape=(dialated_n_steps, inputs[0].shape[1]),
                                          name=scope)(tf.stack(dilated_inputs))


    splitted_outputs = tf.split(dilated_outputs, num_or_size_splits=rate, axis=1)
    # unrolled_outputs = [output for sublist in splitted_outputs for output in sublist]
    unrolled_outputs = tf.unstack(tf.concat(splitted_outputs, axis=0))
    # remove padded zeros
    outputs = unrolled_outputs[:n_steps]

    return outputs
class MyModel(Model):
    def __init__(self,  n_classes, rate):
        super(MyModel, self).__init__()




        self.softmax = tf.keras.activations.softmax
        self.cells = [LSTMCell(50) for _ in range(5)]
        self.rate = rate
        self.classifier = Dense(n_classes, activation='softmax')
        self.dense1 = Dense(128, activation='relu')

        self.flatten=Flatten()

    def call(self, x):
        for i in range(5):
            x = dRNN(self.cells[i], x, 2**i)

        x = self.dense1(x[-1])

        return  self.classifier(x)
from tensorflow.keras.datasets import mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
from tensorflow.keras.utils import to_categorical
train_labels_categorical = to_categorical(train_labels)
test_labels_categorical = to_categorical(test_labels)
# Reshape the data to 2D for SimpleRNN
train_images = train_images.reshape((train_images.shape[0], 28*28,1 ))
test_images = test_images.reshape((test_images.shape[0], 28*28, 1))

# Scale the data
train_images = train_images / 255.0
test_images = test_images / 255.0
def _rnn_reformat(x, input_dims, n_steps):


    x_ = tf.transpose(x, [1, 0, 2])

    x_ = tf.reshape(x_, [-1, input_dims])

    x_reformat = tf.split(x_, n_steps, 0)

    return x_reformat

x_reformat=_rnn_reformat(train_images, 1, 28*28)
x_reformat_test=_rnn_reformat(test_images, 1, 28*28)

from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

model = MyModel( 10, 1)

model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_reformat, train_labels_categorical, batch_size=128, epochs=5, validation_data=(x_reformat_test, test_labels_categorical))
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_labels_categorical = to_categorical(train_labels)
test_labels_categorical = to_categorical(test_labels)
# Reshape the data to 2D for SimpleRNN
train_images = train_images.reshape((train_images.shape[0], 28*28,1 ))
test_images = test_images.reshape((test_images.shape[0], 28*28, 1))

# Scale the data
train_images = train_images / 255.0
test_images = test_images / 255.0
np.random.seed(0)  # Use a fixed seed for reproducibility
permutation = np.random.permutation(train_images.shape[1])

# Apply the permutation to the data
x_train_permuted = train_images[:, permutation,:]
x_test_permuted = test_images[:, permutation,:]
x_reformat=_rnn_reformat(x_train_permuted, 1, 28*28)
x_reformat_test=_rnn_reformat(x_test_permuted, 1, 28*28)
class MyModel(Model):
    def __init__(self,  n_classes, rate):
        super(MyModel, self).__init__()




        self.softmax = tf.keras.activations.softmax
        self.cells = [SimpleRNNCell(20) for _ in range(8)]
        self.rate = rate
        self.classifier = Dense(n_classes, activation='softmax')
        self.dense1 = Dense(128, activation='relu')

        self.flatten=Flatten()

    def call(self, x):
        for i in range(8):
            x = dRNN(self.cells[i], x, 2**i)

        #x = self.dense1(x[-1])

        return  self.classifier(x[-1])

model = MyModel( 10, 1)

model.compile(loss='CategoricalCrossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(x_reformat, train_labels_categorical, batch_size=128, epochs=10, validation_data=(x_reformat_test, test_labels_categorical))
